# Web Scraping with Python

MIT Political Methodology Lab Workshop Series  
Spring 2021  
Andy Halterman

## Scraping Overview

Most news sites and similar web pages can be scraped through a three step process:

1. given a link to an article, extracting and formatting all the needed info from the page
2. given an archive-type page of links, finding all the links on the page.
3. iterating through each page of archives, scraping all the pages from it, and saving to disk

We'll write one function for each.

## Libraries and setup

In [ ]:
# requests is for general HTTP loading
import requests
# BeautifulSoup is an HTML parser
from bs4 import BeautifulSoup
# JSON is a nice format for writing out
import json

In [ ]:
# Sometimes you'll need the regular expressions library and a date library
import re
import dateutil.parser

## Page scraper

We'll write a function here that takes in the URL of an article or page, extracts the information we want from HTML, and structures the output.

Python things to learn:

- calling methods from objects
- what a dictionary looks like
- how to define a function

HTML things to learn: 

- Chrome inspector
- what HTML tags look like

In [ ]:
# find the URL of an article to scrape
url = "https://reliefweb.int/report/afghanistan/more-200-displaced-families-receive-cash-assistance-laghman-province"

In [ ]:
# download article page and get content
page = 
content = 
content

In [ ]:
# convert to BeautifulSoup
soup = 

In [ ]:
# extract body text from page HTML (we'll do this together)
body = 

In [ ]:
# extract title from page HTML
title = 

In [ ]:
# extract author from page HTML
author = 

In [ ]:
# extract date from page HTML
raw_date = 
# convert from raw text into a standardized form
date = dateutil.parser.parse(raw_date)
# Put it into a standard ISO format 
date = date.strftime("%Y-%m-%d")

In [ ]:
# put it all together!
def page_scraper(url):
    """Function to scrape a page"""
    # Code to download and soupify the page
    
    # All the code to extract pieces from the HTML
    title = ""
    ...
    article = {
        "title" : title,
        "body" : body,
        "date" : date,
        "author" : author
    }
    return article

In [ ]:
# test it
page_scraper(url)

In [ ]:
# test it on a different page
page_scraper("https://reliefweb.int/report/iraq/reconstruction-needed-displaced-iraqis-continue-return-iom-iraq")

## Link getter

Second, we need to get the URLS of all the pages we want to scrape. We can do this by finding the directory pages, where the links are on it, and how to get all available directory pages.

Python things to learn here:

- getting values from dictionaries
- for loops and list comprehensions
- regex with the `re` library
- basic string operations

In [ ]:
### Function to get all the article links from a single directory page

In [ ]:
url = "https://reliefweb.int/updates?advanced-search=%28PC13%29_%28F10%29&page=1"
page = 
content = 

In [ ]:
soup = 

In [ ]:
links = soup.find("?????").find_all("a")
print(links[0:5])

In [ ]:
# pull out just the links
links = [i['href'] for i in links]
links[0:10]

In [ ]:
# uh oh! `links` is full of all sorts of garbage. Is there a term
#  that we can search for to reliably pull out article links only?
links = [i for i in links if bool(re.search("?????", i))]
links[0:5]

In [ ]:
# If these weren't complete URLs (a common occurrance) we could use string operations plus a list comprehension to fix this:
#links = ["https://reliefweb.int/" + i for i in links]
#links[0:5]

In [ ]:
# Put it all together into a function that takes in a "page number" 
#  and returns all the links to scrape from it.
def page_to_link(page_num):
    # how to use .format()
    link = "https://reliefweb.int/updates?advanced-search=%28PC13%29_%28F10%29&page={0}".format(page_num)
    # download the page
    # get its content
    # soupify
    # pull out links
    # clean links
    return links

## Putting it together

Now we have a function that'll take a page number for the archive page and return all the links.
We have another function that'll take in an article URL and give us the structured content from the page.

Let's put them together and download a (small!) range of stories.

Note: let's be nice to the UN and not all download the whole thing at once.

In [ ]:
# get all the links we want to scrape
all_links = []

for num in range(1, 5):
    lks = page_to_link(num)
    all_links.extend(lks) # extend! not append.
    
len(all_links)

In [ ]:
all_content = []

for link in all_links[10:20]: # be nice to reliefweb and only get some 
    try:
        content = page_scraper(link)
        all_content.append(content) # back to append!
    except Exception as e:
        # if something goes wrong, keep trucking,
        #  but print out the link so we can diagnose it.
        print(e)
        print(link)

In [ ]:
len(all_content)

In [ ]:
all_content[4]

## Saving as CSV

If you're going to work with your text next in R, a CSV is probably the most useful form to save your text in. To save it as a csv, we will convert into a `DataFrame` using `pandas`, a package for working with data in Python. `pandas` will then let us easily write it out to a CSV.

In [ ]:
import pandas as pd

content_df = pd.DataFrame(all_content)
content_df.to_csv("all_content.csv")

## Saving as JSON

We can also store is as a JSON file. JSON and dictionaries are almost equivalent, so it's a natural form to save a dict as a JSON file.

In [ ]:
import json

In [ ]:
FILENAME = "reliefweb.json"

with open(FILENAME, "w") as f:
    json.dump(all_content, f)

### Read it back in

If you want to load it back in later to analyze, you can do this:

In [ ]:
FILENAME = "reliefweb.json"

with open(FILENAME, "r") as f:
    loaded_content = json.load(f)

In [ ]:
# is it the same?
assert loaded_content[4] == all_content[4]